In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statsmodels.api as sm
from scipy.special import gamma, loggamma, factorial
import scipy.stats
from scipy.interpolate import interp1d
from matplotlib.ticker import AutoMinorLocator 
from matplotlib import rc, font_manager
from matplotlib.lines import Line2D
from matplotlib import colors as mcolors
from mycolorpy import colorlist as mcp
from matplotlib import legend_handler
from matplotlib_scalebar.scalebar import ScaleBar
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
import collections
import matplotlib
import shapely.geometry
from shapely.geometry import Point
from shapely.geometry import Polygon
import shapely.ops 
from geopy.geocoders import Nominatim
import geopandas as gpd
import mapclassify
import jenkspy
import random
import contextily as cx
import os
import rioxarray as rx
import pyreadr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from pysal.lib import weights
from libpysal.io import open as psopen
# import plots
import seaborn as sns
from datetime import datetime, timedelta
import jenkspy
import pyquadkey2
from pyquadkey2.quadkey import QuadKey

In [2]:
wd = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [3]:
grid_pop = gpd.read_file(wd + '/data/output/spain/gridded-geography/gridded_country_shp.shp')

In [4]:
files = sorted(os.listdir(wd + '/data/input/facebook/network-coverage/probability/5225941450021572_2024-10-30_2024-11-14_csv'))
files = [file for file in files]




In [5]:
files

['5225941450021572_2024-10-30_0000.csv',
 '5225941450021572_2024-10-31_0000.csv',
 '5225941450021572_2024-11-01_0000.csv',
 '5225941450021572_2024-11-02_0000.csv',
 '5225941450021572_2024-11-03_0000.csv',
 '5225941450021572_2024-11-04_0000.csv',
 '5225941450021572_2024-11-05_0000.csv',
 '5225941450021572_2024-11-06_0000.csv',
 '5225941450021572_2024-11-07_0000.csv',
 '5225941450021572_2024-11-08_0000.csv',
 '5225941450021572_2024-11-09_0000.csv',
 '5225941450021572_2024-11-10_0000.csv',
 '5225941450021572_2024-11-11_0000.csv',
 '5225941450021572_2024-11-12_0000.csv',
 '5225941450021572_2024-11-13_0000.csv']

In [10]:
df = pd.read_csv(wd + '/data/input/facebook/network-coverage/probability/5225941450021572_2024-10-30_2024-11-14_csv' + '/' + files[7])

df

,p_connectivity,country,lon,lat
0,1.0,ES,2.933350,42.057450
1,1.0,ES,-3.350830,41.812267
2,0.0,ES,0.714111,42.106373
3,1.0,ES,-2.955322,37.935533
4,1.0,ES,-4.383545,41.352072
...,...,...,...,...
29647,1.0,ES,-4.119873,38.711232
29648,1.0,ES,-1.285400,39.410733
29649,1.0,ES,2.977295,39.681825
29650,1.0,ES,-0.384521,39.580290


In [12]:
gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")
        

,p_connectivity,country,lon,lat,geometry
0,1.0,ES,2.933350,42.057450,POINT (2.93335 42.05745)
1,1.0,ES,-3.350830,41.812267,POINT (-3.35083 41.81227)
2,0.0,ES,0.714111,42.106373,POINT (0.71411 42.10637)
3,1.0,ES,-2.955322,37.935533,POINT (-2.95532 37.93553)
4,1.0,ES,-4.383545,41.352072,POINT (-4.38354 41.35207)
...,...,...,...,...,...
29647,1.0,ES,-4.119873,38.711232,POINT (-4.11987 38.71123)
29648,1.0,ES,-1.285400,39.410733,POINT (-1.28540 39.41073)
29649,1.0,ES,2.977295,39.681825,POINT (2.97729 39.68183)
29650,1.0,ES,-0.384521,39.580290,POINT (-0.38452 39.58029)


## Add geometries to network coverage data and save

In [7]:
for i in range(len(files)):   
    
    print(i/len(files)*100)
    
    try:
        
        file = files[i]
        df = pd.read_csv(wd + '/data/input/facebook/network-coverage/probability/5225941450021572_2024-10-30_2024-11-14_csv' + '/' + file)
        gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4258")

    except:
        print(i)
    
    t = file[-19:-4]   
    path = wd + '/data/output/spain/facebook/network-coverage/grid-by-twindow/network-coverage_' + str(t)
    if not os.path.exists(path):
        os.makedirs(path)
    gdf.to_file(path + '/network-coverage_' + str(t) + '.shp')
    
    
    

0.0


/var/folders/79/65l52xsj7vq_4_t_l6k5bl2c0000gn/T/ipykernel_82019/1024092699.py:18: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(path + '/network-coverage_' + str(t) + '.shp')


6.666666666666667


/var/folders/79/65l52xsj7vq_4_t_l6k5bl2c0000gn/T/ipykernel_82019/1024092699.py:18: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(path + '/network-coverage_' + str(t) + '.shp')


13.333333333333334


...

/var/folders/79/65l52xsj7vq_4_t_l6k5bl2c0000gn/T/ipykernel_82019/1024092699.py:18: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(path + '/network-coverage_' + str(t) + '.shp')


86.66666666666667


/var/folders/79/65l52xsj7vq_4_t_l6k5bl2c0000gn/T/ipykernel_82019/1024092699.py:18: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(path + '/network-coverage_' + str(t) + '.shp')


93.33333333333333


/var/folders/79/65l52xsj7vq_4_t_l6k5bl2c0000gn/T/ipykernel_82019/1024092699.py:18: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(path + '/network-coverage_' + str(t) + '.shp')
